In [ ]:
# Tentativa de reformular graph_utils, colocando necessidade de segregar a malha ou não, e identificação de sentido ou não.
import pandas as pd

In [5]:
import hashlib

data = "lucas"
data_bytes = data.encode("utf-8") # converter pra byte
hash_object = hashlib.sha256(data_bytes)
hash_hex = hash_object.hexdigest() # hash em formato hexadecimal
print("Hash SHA-256:", hash_hex)
# hashlib.sha256("Madriles".encode("utf-8")).hexdigest()

Hash SHA-256: 7cadab457ad8d811f134612436daaa5e5914b20dc2502865f714035b0f267680


In [ ]:
def get_projection(main_string, path):
    path_set = set(path)
    main_list = main_string.split()
    result = []
    prev = None

    for i, station in enumerate(main_list):
        if station in path_set:
            if prev is not None and (i == 0 or main_list[i-1] != prev):
                result.append("|")
            result.append(station)
            prev = station

    return result


def find_edge(G, a, b, malha):
    try:
        if G.edges[(a,b)]['ferrovia']==malha:
            return G.edges[(a,b)]['weight']
        else:
            return 0
    except:
        return 0

def find_tku(G, main_string, path, tu, malha):
    km = 0
    stations = get_projection(main_string, path)
    for i in range(len(stations)-1):
        if find_edge(G,stations[i], stations[i+1], malha) is None:
            print(stations[i], stations[i+1], malha)

        km += find_edge(G,stations[i], stations[i+1], malha)

    tku = km * tu
    return tku